In [1]:
# !pip install pandas numpy seaborn scipy openpyxl

In [2]:
import pandas as pd
import numpy as np
import seaborn.objects as so
import seaborn as sns
from scipy import stats
import math
from itertools import product

In [3]:
xl_path = 'Lista de Exercicios - Complementaresxlsx Portugues.xlsx'

# Lista de Exercícios

## Exercício 1

In [4]:
def freq(df: pd.DataFrame, col: str, sort_index=False) -> pd.DataFrame:
    res = (
        df.loc[:,col]
        .value_counts()
        .to_frame(name='cnt')
        .pipe(lambda df: df.sort_index() if sort_index else df)
        .assign(
            relative_freq=lambda df: df.cnt.div(df.cnt.sum()).mul(100),
            cumcnt=lambda df: df.cnt.cumsum(),
            cum_relative_freq=lambda df: df.cumcnt.div(df.cnt.sum()).mul(100),
        )
    )
    return res

In [5]:
(
    pd.read_excel(
        xl_path,
        sheet_name='Exercício 1',
        usecols=[0,1],
        nrows=50,
    )
    .assign(
        bins=lambda df: df['Renda (R$)'].pipe(pd.cut,bins=[0,2000,4000,6000,8000,10000,12000]),
    )
    .pipe(freq,col='bins',sort_index=True)
)

,cnt,relative_freq,cumcnt,cum_relative_freq
bins,,,,
"(0, 2000]",9,18.0,9,18.0
"(2000, 4000]",19,38.0,28,56.0
"(4000, 6000]",11,22.0,39,78.0
"(6000, 8000]",5,10.0,44,88.0
"(8000, 10000]",4,8.0,48,96.0
"(10000, 12000]",2,4.0,50,100.0


---

## Exercício 2

In [6]:
def describe_more(ser: pd.Series, percentiles=[.5]) -> pd.Series:
    res = (    
        ser.describe(percentiles)
        .to_frame()
        .T
        .assign(
            amplitude=ser.max()-ser.min(),
            var=ser.var(),
            stderr=ser.sem(),
            cov=(ser.std()/ser.mean())*100,
            mode=ser.mode().values if len(ser.mode().values) == 1 else np.nan,
            skew=ser.skew(),
            kurtosis=ser.kurt(),
        )
        .T
        .iloc[:,0]
    )
    return res


In [7]:
exerc2_df = (
    pd.read_excel(
        xl_path,
        sheet_name='Exercício 2',
        usecols=[0,1,2],
        nrows=24,
        index_col=0,
    )
)

In [8]:
(
    exerc2_df.T
    .groupby(level=0)
    .apply(lambda df: df.iloc[0].pipe(describe_more,percentiles=[.25,.75,.8,.9,.27,.64]))
    .T
    .round(4)
)

,Ação 1,Ação 2
count,23.0000,23.0000
mean,0.0352,0.1511
std,0.1292,0.2038
min,-0.2018,-0.1490
25%,-0.0612,0.0204
27%,-0.0564,0.0208
50%,0.0370,0.1248
64%,0.0763,0.1825
75%,0.1160,0.2333
80%,0.1260,0.2619


In [9]:
i, j = exerc2_df.shape
ddof = i-j
r, pvalue = stats.pearsonr(*exerc2_df.T.to_numpy())
T = r/math.sqrt((1-(r**2))/(exerc2_df.count().iloc[0]-2))
critical05 = stats.t.ppf(1-.05/2, ddof)  # 2-tail, thus we divide by 2
print(f'''r: {r:.4f}
T: {T:.4f}
pvalue: {pvalue:.4f}
Critical (.05): {critical05:.4f}
''')

r: 0.2908
T: 1.3929
pvalue: 0.1782
Critical (.05): 2.0796



---

## Exercício 7

In [10]:
exerc7_df = (
    pd.read_excel(
        xl_path,
        sheet_name='Exercício 7',
        usecols=[1,2,3],
        nrows=1,
    )
    .T
    .rename(columns={0: 'f_obs'})
    .assign(
        f_exp=lambda df: (df.sum()/len(df.index)).iloc[0]
    )
)
exerc7_df

,f_obs,f_exp
Livro A,29,20.0
Livro B,15,20.0
Livro C,16,20.0


In [11]:
ddof = len(exerc7_df.index)-1
alpha = .05
chi2val, _ = stats.chisquare(exerc7_df.f_obs, exerc7_df.f_exp, ddof=ddof)
pvalue = stats.chi2.sf(chi2val,ddof)
critical05 = stats.chi2.ppf(1-alpha, ddof)

In [12]:
print(f'''χ²: {chi2val:.3f}
pvalue: {pvalue:.4f}
Critical (.05): {critical05:.3f}
''')

χ²: 6.100
pvalue: 0.0474
Critical (.05): 5.991



---

## Exercício 8

In [13]:
exerc8_df = (
    pd.read_excel(
        xl_path,
        sheet_name='Exercício 8',
        usecols=[0,1],
        nrows=14,
    )
)

In [14]:
alpha = .05
fvar = exerc8_df.var()['Local A (mm)']/exerc8_df.var()['Local B (mm)']
pvalue = stats.f.sf(fvar, 13, 13)
critical05 = stats.f.ppf(1-alpha, 13, 13)

In [15]:
print(f'''F var: {fvar:.3f}
pvalue: {pvalue:.5f}
Critical (.05): {critical05:.3f}
''')

F var: 2.761
pvalue: 0.03917
Critical (.05): 2.577

